Import Module

In [1]:
from tensorflow.keras.layers import Dense, Flatten, Activation, GlobalAveragePooling2D
from tensorflow.keras.datasets.cifar100 import load_data
from tensorflow.keras.applications import MobileNetV2, ResNet50, InceptionV3, VGG16, EfficientNetB0, DenseNet121, Xception, NASNetMobile, InceptionResNetV2, EfficientNetV2B0
from tensorflow.keras.models import Model;
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np
import random

2025-05-24 14:33:42.227532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748075622.379968   11069 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748075622.423615   11069 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748075622.766846   11069 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748075622.766894   11069 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748075622.766934   11069 computation_placer.cc:177] computation placer alr

In [2]:
SEED =  50
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
(x_train, y_train), (x_test, y_test) = load_data(label_mode='fine')

In [4]:
classes = random.sample(range(100), 20)

In [5]:
def filter_classes(x,y,selected):
    x_filtered = []
    y_filtered = []
    class_map = {cls: idx for idx, cls in enumerate(selected)}
    for img, label in zip(x,y):
        if label[0] in selected:
            x_filtered.append(img)
            y_filtered.append(class_map[label[0]])
    return np.array(x_filtered), np.array(y_filtered)

In [6]:
x_train, y_train = filter_classes(x_train, y_train, classes)
x_test, y_test = filter_classes(x_test, y_test, classes)

In [7]:
def preprocess_data(x,y):
    x = tf.image.resize(x,(224,224))/255.0
    y = to_categorical(y, num_classes=20)
    return x,y

In [8]:
x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

W0000 00:00:1748075649.510175   11069 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-05-24 14:34:09.560180: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6021120000 exceeds 10% of free system memory.
2025-05-24 14:34:14.471137: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6021120000 exceeds 10% of free system memory.
2025-05-24 14:34:17.213520: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1204224000 exceeds 10% of free system memory.
2025-05-24 14:34:18.262372: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1204224000 exceeds 10% of free system memory.


1. MobileNetV2

In [9]:
model_0 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
model_0.trainable = False # Freeze base layers

In [10]:
inputs = model_0.input
output = model_0.output
x = GlobalAveragePooling2D()(output)
outputs = Dense(20, activation='softmax')(x)
model = Model(inputs, outputs)

In [12]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 141s 554ms/step - accuracy: 0.1589 - loss: 2.7636 - val_accuracy: 0.4815 - val_loss: 1.8746
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 142s 567ms/step - accuracy: 0.5297 - loss: 1.6899 - val_accuracy: 0.6195 - val_loss: 1.3779
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 137s 547ms/step - accuracy: 0.6502 - loss: 1.2743 - val_accuracy: 0.6655 - val_loss: 1.1616
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 135s 541ms/step - accuracy: 0.6860 - loss: 1.0786 - val_accuracy: 0.6910 - val_loss: 1.0443
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 135s 541ms/step - accuracy: 0.7096 - loss: 0.9651 - val_accuracy: 0.7070 - val_loss: 0.9701


In [13]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy for MobileNetV2: {acc*100:.2f}%")

63/63 ━━━━━━━━━━━━━━━━━━━━ 27s 431ms/step - accuracy: 0.7066 - loss: 0.9543
Test Accuracy for MobileNetV2: 69.80%


ResNet50

2. Pretrained model ResNet50

In [14]:
model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
model_resnet.trainable = False

In [15]:
inputs = model_resnet.input
outputs = model_resnet.output

x = GlobalAveragePooling2D()(outputs)
outputs = Dense(20, activation='softmax')(x)
model = Model(inputs, outputs)

In [16]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split=0.2)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 396s 2s/step - accuracy: 0.0494 - loss: 3.0520 - val_accuracy: 0.0525 - val_loss: 2.9954
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 386s 2s/step - accuracy: 0.0559 - loss: 2.9940 - val_accuracy: 0.0550 - val_loss: 2.9915
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 385s 2s/step - accuracy: 0.0588 - loss: 2.9897 - val_accuracy: 0.0555 - val_loss: 2.9877
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 385s 2s/step - accuracy: 0.0630 - loss: 2.9855 - val_accuracy: 0.0565 - val_loss: 2.9840
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 392s 2s/step - accuracy: 0.0677 - loss: 2.9814 - val_accuracy: 0.0560 - val_loss: 2.9803


In [17]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy for ResNet50: {acc*100:.2f}%")

63/63 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.0598 - loss: 2.9757
Test Accuracy for ResNet50: 5.65%


3. VGG16 model

In [18]:
model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
model_vgg16.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step


In [19]:
inputs = model_vgg16.input
outputs = model_vgg16.output
x = GlobalAveragePooling2D()(outputs)
outputs = Dense(20, activation='softmax')(x)
model = Model(inputs, outputs)

In [20]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split = 0.2)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 972s 4s/step - accuracy: 0.0509 - loss: 3.1013 - val_accuracy: 0.0470 - val_loss: 2.9597
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 960s 4s/step - accuracy: 0.0882 - loss: 2.9465 - val_accuracy: 0.1675 - val_loss: 2.9100
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 962s 4s/step - accuracy: 0.1821 - loss: 2.8975 - val_accuracy: 0.2035 - val_loss: 2.8709
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 951s 4s/step - accuracy: 0.2223 - loss: 2.8573 - val_accuracy: 0.2290 - val_loss: 2.8337
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 919s 4s/step - accuracy: 0.2398 - loss: 2.8196 - val_accuracy: 0.2480 - val_loss: 2.7986


In [21]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy for VGG16: {acc*100:.2f}%")

63/63 ━━━━━━━━━━━━━━━━━━━━ 183s 3s/step - accuracy: 0.2476 - loss: 2.7937
Test Accuracy for VGG16: 25.80%


4. EfficientNetB0 Model

In [22]:
model_efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
model_efficientnet.trainable = False

In [23]:
inputs = model_efficientnet.input
outputs = model_efficientnet.output
x = GlobalAveragePooling2D()(outputs)
outputs = Dense(20, activation='softmax')(x)
model = Model(inputs, outputs)

In [24]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split = 0.2)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 211s 825ms/step - accuracy: 0.0489 - loss: 3.0101 - val_accuracy: 0.0525 - val_loss: 3.0005
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 205s 822ms/step - accuracy: 0.0521 - loss: 3.0002 - val_accuracy: 0.0525 - val_loss: 3.0001
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 205s 821ms/step - accuracy: 0.0548 - loss: 2.9999 - val_accuracy: 0.0525 - val_loss: 3.0005
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 218s 872ms/step - accuracy: 0.0503 - loss: 2.9995 - val_accuracy: 0.0525 - val_loss: 3.0004
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 208s 830ms/step - accuracy: 0.0491 - loss: 3.0002 - val_accuracy: 0.0525 - val_loss: 3.0001


In [25]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy for EfficientNetB0: {acc*100:.2f}%")

63/63 ━━━━━━━━━━━━━━━━━━━━ 41s 643ms/step - accuracy: 0.0528 - loss: 2.9970
Test Accuracy for EfficientNetB0: 5.00%


5. DenseNet121 Model

In [26]:
model_densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))
model_densenet.trainable = False

In [27]:
inputs = model_densenet.input
outputs = model_densenet.output
x = GlobalAveragePooling2D()(outputs)
outputs = Dense(20, activation='softmax')(x)
model = Model(inputs, outputs)

In [28]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split = 0.2)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 433s 2s/step - accuracy: 0.0696 - loss: 3.2561 - val_accuracy: 0.2720 - val_loss: 2.4387
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 425s 2s/step - accuracy: 0.3429 - loss: 2.3053 - val_accuracy: 0.4885 - val_loss: 1.8874
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 425s 2s/step - accuracy: 0.5201 - loss: 1.8106 - val_accuracy: 0.5770 - val_loss: 1.5658
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 402s 2s/step - accuracy: 0.6045 - loss: 1.5159 - val_accuracy: 0.6320 - val_loss: 1.3678
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 401s 2s/step - accuracy: 0.6504 - loss: 1.3294 - val_accuracy: 0.6595 - val_loss: 1.2363


In [29]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy for DenseNet121: {acc*100:.2f}%")

63/63 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.6493 - loss: 1.2759
Test Accuracy for DenseNet121: 66.05%


6. NASNetMobile Model

In [30]:
model_nasnet = NASNetMobile(weights='imagenet', include_top=False, input_shape=(224,224,3))
model_nasnet.trainable = False

In [31]:
inputs = model_nasnet.input
outputs = model_nasnet.output
x = GlobalAveragePooling2D()(outputs)
outputs = Dense(20, activation='softmax')(x)
model = Model(inputs, outputs)

In [32]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split = 0.2)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 174s 651ms/step - accuracy: 0.1883 - loss: 2.7652 - val_accuracy: 0.5370 - val_loss: 1.9510
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 149s 595ms/step - accuracy: 0.5900 - loss: 1.7513 - val_accuracy: 0.6660 - val_loss: 1.3989
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 148s 594ms/step - accuracy: 0.6887 - loss: 1.2889 - val_accuracy: 0.7110 - val_loss: 1.1453
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 149s 596ms/step - accuracy: 0.7213 - loss: 1.0684 - val_accuracy: 0.7270 - val_loss: 1.0111
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 149s 597ms/step - accuracy: 0.7413 - loss: 0.9451 - val_accuracy: 0.7365 - val_loss: 0.9291


In [33]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy for NASNetMobile: {acc*100:.2f}%")

63/63 ━━━━━━━━━━━━━━━━━━━━ 30s 477ms/step - accuracy: 0.7293 - loss: 0.9526
Test Accuracy for NASNetMobile: 72.85%


7. EfficientNetV2B0

In [34]:
model_efficientv2 = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(224,224,3))
model_efficientv2.trainable = False

In [35]:
inputs = model_efficientv2.input
outputs = model_efficientv2.output
x = GlobalAveragePooling2D()(outputs)
outputs = Dense(20, activation='softmax')(x)
model = Model(inputs, outputs)

In [36]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split = 0.2)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 153s 586ms/step - accuracy: 0.0499 - loss: 3.0135 - val_accuracy: 0.0525 - val_loss: 3.0001
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 129s 518ms/step - accuracy: 0.0528 - loss: 3.0014 - val_accuracy: 0.0525 - val_loss: 2.9998
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 131s 524ms/step - accuracy: 0.0505 - loss: 3.0009 - val_accuracy: 0.0525 - val_loss: 2.9997
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 130s 521ms/step - accuracy: 0.0496 - loss: 3.0002 - val_accuracy: 0.0525 - val_loss: 2.9994
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 129s 517ms/step - accuracy: 0.0529 - loss: 3.0009 - val_accuracy: 0.0525 - val_loss: 2.9995


In [37]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy for EfficientNetV2B0: {acc*100:.2f}%")

63/63 ━━━━━━━━━━━━━━━━━━━━ 26s 406ms/step - accuracy: 0.0528 - loss: 2.9967
Test Accuracy for EfficientNetV2B0: 5.00%


8. InceptionV3

In [38]:
model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))
model_inception.trainable = False

In [39]:
inputs = model_inception.input
outputs = model_inception.output
x = GlobalAveragePooling2D()(outputs)
outputs = Dense(20, activation='softmax')(x)
model = Model(inputs, outputs)

In [42]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split = 0.2)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 257s 1s/step - accuracy: 0.6868 - loss: 1.2324 - val_accuracy: 0.7200 - val_loss: 1.0238
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 247s 988ms/step - accuracy: 0.7456 - loss: 0.9297 - val_accuracy: 0.7480 - val_loss: 0.8917
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 206s 824ms/step - accuracy: 0.7714 - loss: 0.8017 - val_accuracy: 0.7620 - val_loss: 0.8226
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 206s 825ms/step - accuracy: 0.7849 - loss: 0.7240 - val_accuracy: 0.7680 - val_loss: 0.7792
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 207s 827ms/step - accuracy: 0.7970 - loss: 0.6689 - val_accuracy: 0.7730 - val_loss: 0.7490


In [43]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy for InceptionV3: {acc*100:.2f}%")

63/63 ━━━━━━━━━━━━━━━━━━━━ 42s 656ms/step - accuracy: 0.7564 - loss: 0.7832
Test Accuracy for InceptionV3: 74.85%


9. Xception

In [44]:
model_xception = Xception(weights='imagenet', include_top=False, input_shape=(224,224,3))
model_xception.trainable = False

In [45]:
inputs = model_xception.input
outputs = model_xception.output
x = GlobalAveragePooling2D()(outputs)
outputs = Dense(20, activation='softmax')(x)
model = Model(inputs, outputs)

In [46]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split = 0.2)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 502s 2s/step - accuracy: 0.2372 - loss: 2.7119 - val_accuracy: 0.6605 - val_loss: 1.7473
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 438s 2s/step - accuracy: 0.6956 - loss: 1.5786 - val_accuracy: 0.7340 - val_loss: 1.2221
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 494s 2s/step - accuracy: 0.7439 - loss: 1.1474 - val_accuracy: 0.7550 - val_loss: 1.0054
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 440s 2s/step - accuracy: 0.7632 - loss: 0.9570 - val_accuracy: 0.7660 - val_loss: 0.8934
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 440s 2s/step - accuracy: 0.7771 - loss: 0.8510 - val_accuracy: 0.7720 - val_loss: 0.8251


In [47]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy for Xception: {acc*100:.2f}%")

63/63 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.7744 - loss: 0.8275
Test Accuracy for Xception: 76.70%


10. InceptionResNetV2

In [48]:
model_inceptionv2 = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
model_inceptionv2.trainable = False

In [49]:
inputs = model_inceptionv2.input
outputs = model_inceptionv2.output
x = GlobalAveragePooling2D()(outputs)
outputs = Dense(20, activation='softmax')(x)
model = Model(inputs, outputs)

In [50]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split = 0.2)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 483s 2s/step - accuracy: 0.2712 - loss: 2.5983 - val_accuracy: 0.7035 - val_loss: 1.2580
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 465s 2s/step - accuracy: 0.7224 - loss: 1.1417 - val_accuracy: 0.7565 - val_loss: 0.8808
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 465s 2s/step - accuracy: 0.7731 - loss: 0.8368 - val_accuracy: 0.7780 - val_loss: 0.7523
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 465s 2s/step - accuracy: 0.7937 - loss: 0.7158 - val_accuracy: 0.7885 - val_loss: 0.6858
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 515s 2s/step - accuracy: 0.8091 - loss: 0.6466 - val_accuracy: 0.8005 - val_loss: 0.6439


In [51]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy for InceptionResNetV2: {acc*100:.2f}%")

63/63 ━━━━━━━━━━━━━━━━━━━━ 93s 1s/step - accuracy: 0.7894 - loss: 0.6656
Test Accuracy for InceptionResNetV2: 79.30%


Clear GPU Memory

In [1]:
from tensorflow.keras import backend as K
import gc

K.clear_session()
gc.collect()


2025-05-24 13:58:14.228861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748073494.246274   60202 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748073494.252261   60202 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748073494.265016   60202 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748073494.265032   60202 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748073494.265034   60202 computation_placer.cc:177] computation placer alr

0

In [10]:
source tf-env/bin/activate

SyntaxError: invalid syntax (29176056.py, line 1)